---
### **CHIRPS**

Download archive of global daily data at 0.25 / .05 degree resolution (only needs to be run once):
```
! cd /rds/general/project/wwa/live/data/chirps_05; for y in `seq 1981 2022`; do wget https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/netcdf/p05/chirps-v2.0.$y.days_p05.nc; done
! cd /rds/general/project/wwa/live/data/chirps_25; for y in `seq 1981 2022`; do wget https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/netcdf/p25/chirps-v2.0.$y.days_p5.nc; done
```

In [ ]:
# update this year's data:

! cd /rds/general/project/wwa/live/data/chirps_05; rm chirps-v2.0.2023.days_p05.nc; wget https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/netcdf/p05/chirps-v2.0.2023.days_p05.nc
! cd /rds/general/project/wwa/live/data/chirps_25; rm chirps-v2.0.2023.days_p25.nc; wget https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/netcdf/p25/chirps-v2.0.2023.days_p25.nc

---
### **CORDEX (and other ESGF data)**



#### **Searching & installing data**
Always search for the data before installing, to make sure you're picking up the right files. Examples of valid searches:
```
synda search domain="AFR-44 AFR-22" variable=pr
synda search domain=EUR-11 variable=tas
```
Once you're happy with the search criteria, you can install the data by replacing 'search' with 'install'.

After you're finished with the data, it's a good idea to uninstall it by running the same command, but replacing 'search' with 'remove'.


#### **Initial setup**

Any data that is available from ESGF can be downloaded via `synda`. To use synda you'll need a dedicated conda environment, created via
```
! conda create -y --name synda -c IPSL synda
```
You will also need to declare the `ST_HOME` variable in your .bashrc file, by running
```
cat `export ST_HOME=/rds/general/user/cb2714/projects/wwa/ephemeral/synda` >> .bashrc
```

Then run `conda activate synda` and you should be ready to start downloading.

#### **Re-initialising the synda directory**

Synda is set up to run on the ephemeral drive, so data will be deleted after 30 days.  
Depending on how much of the file setup is deleted, the synda file tree may need to be reinitialised before new data can be downloaded. To do this, run `synda init-env` and enter `y` to create new file stubs.  
Someone will need to provide their ESGF credentials: to download CORDEX data, membership of the CORDEX access control group must be [requested via ESGF](http://esg-dn1.nsc.liu.se/ac/subscribe/CORDEX_Research/).

Before attempting to download data, the default settings need to be modified (otherwise downloads seem to fail for some reason). Open `/rds/general/user/cb2714/projects/wwa/ephemeral/synda/conf/default/default.txt` and replace the contents with the following:
```
# $ST_HOME/conf/defaults/default.txt

latest=true
local_path_format=custom
local_path_drs_template=%(project)s/%(domain)s/%(driving_model)s/%(ensemble)s/%(rcm_name)s/%(variable)s

frequency=day
experiment=rcp85 historical

#distrib=true
protocol=http
#replica=true

timeslice=19000101-20501231
```

To search for data from projects other than CORDEX, you may need to un-comment `distrib=true` and/or `replica=true`. You may also need to modify the format of `local_path_drs_template` to remove references to the domain and rcm_name, neither of which are valid fields for CMIP5 or CMIP6.


---
### **CPC**

Download archive of global daily precip, tmax, tmin (only needs to be run once):
```
! cd /rds/general/project/wwa/live/data/cpc; for y in `seq 1979 2023`; do wget https://downloads.psl.noaa.gov/Datasets/cpc_global_precip/precip.$y.nc; wget https://downloads.psl.noaa.gov/Datasets/cpc_global_temp/tmax.$y.nc; wget https://downloads.psl.noaa.gov/Datasets/cpc_global_temp/tmin.$y.nc; done
```

In [ ]:
! cd /rds/general/project/wwa/live/data/cpc; for y in `seq 1979 2023`; do wget https://downloads.psl.noaa.gov/Datasets/cpc_global_precip/precip.$y.nc; wget https://downloads.psl.noaa.gov/Datasets/cpc_global_temp/tmax.$y.nc; wget https://downloads.psl.noaa.gov/Datasets/cpc_global_temp/tmin.$y.nc; done

In [ ]:
# update this year's data:

! cd /rds/general/project/wwa/live/data/cpc; rm precip.2023.nc; rm tmax.2023.nc; rm tmin.2023.nc; wget https://downloads.psl.noaa.gov/Datasets/cpc_global_precip/precip.2023.nc; wget https://downloads.psl.noaa.gov/Datasets/cpc_global_temp/tmax.2023.nc; wget https://downloads.psl.noaa.gov/Datasets/cpc_global_temp/tmin.2023.nc

---
### **MSWEP**

#### **Initial setup**

Data has to be downloaded from Google drive via `rclone`: this requires some initial setup.

The process is slightly more complicated because we're working on a remote machine and can't just authorise access to Google Drive as you usually would.  
First you need to configure `rclone` on your local machine, then copy the config file to the RDS (following method 2 [here](https://rclone.org/remote_setup/)).

First [install rclone](https://rclone.org/install/) following the instructions for your OS.  

Then configure `rclone` on your local machine (walkthrough video available [here](http://www.gloh2o.org/mswep/#faq)):
```
> rclone config
- 'n' to create a new connection - give it a meaningful name (eg mswep)
- storage type: drive
- most options can be left blank as default, unless noted otherwise below
- read-only access
- use auto config: y
- authenticate your Google account
```
When local setup is complete, run `rclone config file` to find out where the config file is saved. We now need to copy this to the RDS.

On the rds, run `rclone config file` to find out where the config file needs to be saved. Paste the config file into this location. (You might need to create an empty text file, paste in the file contents, rename & move to the final location)


#### **To update with the latest data:**
Rclone checks each file to see if the original has been updated since the data was last downloaded.  
Checking every file in the 'past' archive can be quite slow, so it is recommended to do this only once every few months, as data moves from 'nrt' to 'past'.
```
# ! module load rclone; rclone sync -v --drive-shared-with-me mswep:/MSWEP_V280/Past/Daily /rds/general/project/wwa/live/data/mswep_past
! module load rclone; rclone sync -v --drive-shared-with-me mswep:/MSWEP_V280/NRT/Daily /rds/general/project/wwa/live/data/mswep_nrt
```
